In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr


In [2]:
df_grace = pd.read_csv('../../Data/interpolated_lwe_grid_resolution5.csv')
df_grace = df_grace.rename(columns={'lon_centroid': 'lon', 'lat_centroid': 'lat', 'lwe_thickness': 'waterstorage'})
df_grace['lon'] = df_grace['lon'].apply(lambda x: x - 360 if x > 180 else x) 


In [3]:


import pandas as pd
import xarray as xr
import numpy as np
import os # Added for output directory creation

# --- Configuration ---
# Assuming df is your loaded DataFrame before this script runs
# Example:
# df = pd.read_csv('your_input_file.csv')

output_dir = '../../output' # Define output directory
output_csv_file = os.path.join(output_dir, 'downsampled_mascons_5deg_water_aligned.csv')
output_parquet_file = os.path.join(output_dir, 'downsampled_mascons_5deg_water_aligned.parquet')

df = df_grace
# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


# Define the input grid resolution (used for consistency check, not directly in groupby_bins)
lat_res_in = 5
lon_res_in = 5
# Define the desired output grid resolution
lat_res_out = 5.0 # Use float for calculations
lon_res_out = 5.0 # Use float for calculations
# --- End Configuration ---

print(f"Starting script...")
print(f"Output CSV: {output_csv_file}")
print(f"Output Parquet: {output_parquet_file}")
print(f"Input resolution: {lat_res_in}° x {lon_res_in}° (assumed)")
print(f"Output resolution: {lat_res_out}° x {lon_res_out}°")

# --- Step 1: Read the CSV data using Pandas ---
# Assume 'df' is already loaded and contains 'time', 'lat', 'lon', 'waterstorage'
print(f"\nUsing pre-loaded DataFrame...")
try:
    # Make sure columns exist before proceeding
    required_cols = ['time', 'lat', 'lon', 'waterstorage']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"Missing required columns: {missing}")

    # Optional: Convert 'time' column to datetime objects if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df['time']):
        print("Converting 'time' column to datetime...")
        df['time'] = pd.to_datetime(df['time'])

    print("DataFrame ready.")
    print("DataFrame head:")
    print(df.head())

except NameError:
    print("\n---------------------------------------------------------")
    print("Error: DataFrame 'df' not found.")
    print("Please ensure the DataFrame is loaded before running this script.")
    print("---------------------------------------------------------")
    exit()
except ValueError as e:
    print(f"\nError: {e}")
    exit()
except MemoryError: # Keep memory error handling
    print("\n---------------------------------------------------------")
    print("MemoryError: The DataFrame is too large.")
    print("Consider using Dask or chunking if conversion fails later.")
    print("---------------------------------------------------------")
    exit() # Exit the script if memory error occurs
except Exception as e:
     print(f"\nAn unexpected error occurred during DataFrame preparation: {e}")
     exit()


# --- Step 2: Convert Pandas DataFrame to Xarray Dataset ---
print("\nConverting DataFrame to Xarray Dataset...")
try:
    # Check for duplicate indices *before* setting index
    duplicates = df.duplicated(subset=['time', 'lat', 'lon']).sum()
    if duplicates > 0:
        print(f"\nWarning: Found {duplicates} duplicate time/lat/lon combinations. Keeping first occurrence.")
        # Optionally, decide how to handle duplicates (e.g., mean, keep first/last)
        # df = df.groupby(['time', 'lat', 'lon']).mean().reset_index() # Example: average duplicates
        df = df.drop_duplicates(subset=['time', 'lat', 'lon'], keep='first')

    df = df.set_index(['time', 'lat', 'lon'])
    ds = df.to_xarray()

    # Ensure latitude coordinates are sorted (ascending or descending)
    # groupby_bins requires monotonic coordinates
    if not ds.indexes['lat'].is_monotonic_increasing and not ds.indexes['lat'].is_monotonic_decreasing:
         print("Sorting latitude coordinates...")
         ds = ds.sortby('lat')
    if not ds.indexes['lon'].is_monotonic_increasing and not ds.indexes['lon'].is_monotonic_decreasing:
         print("Sorting longitude coordinates...")
         ds = ds.sortby('lon')

    # Optional: Reindex latitude to be decreasing if needed (common standard)
    # if ds['lat'][0] < ds['lat'][-1]:
    #     print("Reindexing latitude to decreasing order...")
    #     ds = ds.reindex(lat=list(reversed(ds['lat'])))

    print("Xarray Dataset created successfully:")
    print(ds)
except KeyError as e:
    print(f"\nError: Column '{e}' not found in DataFrame. Needed for setting index.")
    print("Please ensure your DataFrame has 'time', 'lat', 'lon' columns.")
    exit()
except Exception as e:
    print(f"\nAn error occurred during DataFrame to Xarray conversion: {e}")
    exit()


# --- Step 3: Perform Spatial Aggregation using groupby_bins ---
print(f"\nAggregating data to {lat_res_out}° x {lon_res_out}° grid using groupby_bins...")

try:
    # 3a: Determine the overall extent and create output grid *boundaries*
    min_lat, max_lat = ds['lat'].min().item(), ds['lat'].max().item()
    min_lon, max_lon = ds['lon'].min().item(), ds['lon'].max().item()

    # Calculate the boundaries aligned with the output resolution
    # Use floor for min edge, ceil for max edge, ensuring they align with the grid
    # Add a small epsilon to max extent before ceil to handle points exactly on boundary
    epsilon = 1e-9
    lat_min_bnd = np.floor(min_lat / lat_res_out) * lat_res_out
    lat_max_bnd = np.ceil((max_lat + epsilon) / lat_res_out) * lat_res_out
    lon_min_bnd = np.floor(min_lon / lon_res_out) * lon_res_out
    lon_max_bnd = np.ceil((max_lon + epsilon) / lon_res_out) * lon_res_out

    # Create the bin edges for grouping
    # Use arange up to max_bnd + resolution/2 to ensure the last bin edge is included
    lat_bins = np.arange(lat_min_bnd, lat_max_bnd + lat_res_out / 2, lat_res_out)
    lon_bins = np.arange(lon_min_bnd, lon_max_bnd + lon_res_out / 2, lon_res_out)

    # 3b: Calculate the desired output grid *centroids* (labels for the bins)
    # These will become the new coordinates in the aggregated dataset
    lat_centroids = lat_bins[:-1] + lat_res_out / 2.0
    lon_centroids = lon_bins[:-1] + lon_res_out / 2.0

    print(f"Latitude bins: {lat_bins}")
    print(f"Latitude centroids: {lat_centroids}")
    print(f"Longitude bins: {lon_bins}")
    print(f"Longitude centroids: {lon_centroids}")

    # 3c: Perform the aggregation
    # Group by latitude bins first, calculate the mean for each bin,
    # then group the result by longitude bins and calculate the mean.
    # Use the calculated centroids as the coordinate labels for the resulting bins.
    # 'right=False' means bins are [left, right), matching np.floor logic.
    ds_agg = ds.groupby_bins('lat', bins=lat_bins, labels=lat_centroids, right=False).mean()
    ds_agg = ds_agg.groupby_bins('lon', bins=lon_bins, labels=lon_centroids, right=False).mean()

    # Rename the coordinate dimensions from 'lat_bins'/'lon_bins' back to 'lat'/'lon'
    ds_agg = ds_agg.rename({'lat_bins': 'lat', 'lon_bins': 'lon'})

    print("Aggregation complete.")
    print("Aggregated Dataset:")
    print(ds_agg)

except Exception as e:
    print(f"\nAn error occurred during aggregation with groupby_bins: {e}")
    print("Check if latitude/longitude coordinates are monotonic (sorted).")
    exit()


# --- Step 4: Convert back to DataFrame and save ---
print("\nConverting aggregated Dataset back to DataFrame...")
# Use .reset_index() to turn the coordinates ('time', 'lat', 'lon')
# back into columns for the CSV/Parquet output.
df_agg = ds_agg.to_dataframe().reset_index()

# Optional: Drop rows where the aggregated value is NaN
# This happens if an output grid cell contained no input data points.
nan_count_before = df_agg['waterstorage'].isnull().sum()
if nan_count_before > 0:
    print(f"Dropping {nan_count_before} rows with NaN 'waterstorage' values (empty grid cells).")
    df_agg = df_agg.dropna(subset=['waterstorage'])

print(f"\nSaving aggregated data...")
try:
    # Save to CSV
    print(f"Saving to CSV: {output_csv_file}")
    df_agg.to_csv(output_csv_file, index=False, float_format='%.5f') # Control float precision

    # Save to Parquet (often more efficient for large data)
    print(f"Saving to Parquet: {output_parquet_file}")
    df_agg.to_parquet(output_parquet_file, index=False)

    df_grace = df_agg.copy()

    print("Aggregated data saved successfully.")
    print("\nFinal DataFrame snippet:")
    print(df_agg.head())

except Exception as e:
    print(f"\nAn error occurred while saving the output files: {e}")

print("\nScript finished.")

Starting script...
Output CSV: ../../output/downsampled_mascons_5deg_water_aligned.csv
Output Parquet: ../../output/downsampled_mascons_5deg_water_aligned.parquet
Input resolution: 5° x 5° (assumed)
Output resolution: 5.0° x 5.0°

Using pre-loaded DataFrame...
Converting 'time' column to datetime...
DataFrame ready.
DataFrame head:
        time     lon    lat  waterstorage
0 2002-04-01 -127.25  47.75      0.000000
1 2002-04-01 -122.25  47.75      2.780771
2 2002-04-01 -117.25  47.75      7.942758
3 2002-04-01 -112.25  47.75      7.106880
4 2002-04-01 -107.25  47.75      4.373613

Converting DataFrame to Xarray Dataset...
Xarray Dataset created successfully:
<xarray.Dataset> Size: 176kB
Dimensions:       (time: 272, lat: 5, lon: 16)
Coordinates:
  * time          (time) datetime64[ns] 2kB 2002-04-01 2002-05-01 ... 2024-11-01
  * lat           (lat) float64 40B 47.75 52.75 57.75 62.75 67.75
  * lon           (lon) float64 128B -127.2 -122.2 -117.2 ... -57.25 -52.25
Data variables:
    wa

In [4]:
df_gldas = pd.read_csv('../../Data/data_GLDAS/compiled_canada_soil_moisture.csv')
print(df_gldas.columns)
# df = df.drop(['SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'], axis = 1)


df_gldas['waterstorage'] = (df_gldas['SoilMoi0_10cm_inst'] * 10 + df_gldas['SoilMoi10_40cm_inst'] * 30 +df_gldas['SoilMoi40_100cm_inst'] * 60 + df_gldas['SoilMoi100_200cm_inst'] * 100) /200
# rename the column 'SoilMoi0_10cm_inst' to 'waterstorage'
# df = df.rename(columns={'SoilMoi0_10cm_inst': 'waterstorage'})


Index(['time', 'lat', 'lon', 'SoilMoi0_10cm_inst', 'SoilMoi10_40cm_inst',
       'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'],
      dtype='object')


In [5]:


import pandas as pd
import xarray as xr
import numpy as np
import os # Added for output directory creation

# --- Configuration ---
# Assuming df is your loaded DataFrame before this script runs
# Example:
# df = pd.read_csv('your_input_file.csv')

output_dir = '../../output' # Define output directory
output_csv_file = os.path.join(output_dir, 'downsampled_GLDAS_5deg_water_aligned.csv')
output_parquet_file = os.path.join(output_dir, 'downsampled_GLDAS_5deg_water_aligned.parquet')

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


# Define the input grid resolution (used for consistency check, not directly in groupby_bins)
lat_res_in = 0.25
lon_res_in = 0.25
# Define the desired output grid resolution
lat_res_out = 5.0 # Use float for calculations
lon_res_out = 5.0 # Use float for calculations
df = df_gldas
# --- End Configuration ---

print(f"Starting script...")
print(f"Output CSV: {output_csv_file}")
print(f"Output Parquet: {output_parquet_file}")
print(f"Input resolution: {lat_res_in}° x {lon_res_in}° (assumed)")
print(f"Output resolution: {lat_res_out}° x {lon_res_out}°")

# --- Step 1: Read the CSV data using Pandas ---
# Assume 'df' is already loaded and contains 'time', 'lat', 'lon', 'waterstorage'
print(f"\nUsing pre-loaded DataFrame...")
try:
    # Make sure columns exist before proceeding
    required_cols = ['time', 'lat', 'lon', 'waterstorage']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"Missing required columns: {missing}")

    # Optional: Convert 'time' column to datetime objects if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df['time']):
        print("Converting 'time' column to datetime...")
        df['time'] = pd.to_datetime(df['time'])

    print("DataFrame ready.")
    print("DataFrame head:")
    print(df.head())

except NameError:
    print("\n---------------------------------------------------------")
    print("Error: DataFrame 'df' not found.")
    print("Please ensure the DataFrame is loaded before running this script.")
    print("---------------------------------------------------------")
    exit()
except ValueError as e:
    print(f"\nError: {e}")
    exit()
except MemoryError: # Keep memory error handling
    print("\n---------------------------------------------------------")
    print("MemoryError: The DataFrame is too large.")
    print("Consider using Dask or chunking if conversion fails later.")
    print("---------------------------------------------------------")
    exit() # Exit the script if memory error occurs
except Exception as e:
     print(f"\nAn unexpected error occurred during DataFrame preparation: {e}")
     exit()


# --- Step 2: Convert Pandas DataFrame to Xarray Dataset ---
print("\nConverting DataFrame to Xarray Dataset...")
try:
    # Check for duplicate indices *before* setting index
    duplicates = df.duplicated(subset=['time', 'lat', 'lon']).sum()
    if duplicates > 0:
        print(f"\nWarning: Found {duplicates} duplicate time/lat/lon combinations. Keeping first occurrence.")
        # Optionally, decide how to handle duplicates (e.g., mean, keep first/last)
        # df = df.groupby(['time', 'lat', 'lon']).mean().reset_index() # Example: average duplicates
        df = df.drop_duplicates(subset=['time', 'lat', 'lon'], keep='first')

    df = df.set_index(['time', 'lat', 'lon'])
    ds = df.to_xarray()

    # Ensure latitude coordinates are sorted (ascending or descending)
    # groupby_bins requires monotonic coordinates
    if not ds.indexes['lat'].is_monotonic_increasing and not ds.indexes['lat'].is_monotonic_decreasing:
         print("Sorting latitude coordinates...")
         ds = ds.sortby('lat')
    if not ds.indexes['lon'].is_monotonic_increasing and not ds.indexes['lon'].is_monotonic_decreasing:
         print("Sorting longitude coordinates...")
         ds = ds.sortby('lon')

    # Optional: Reindex latitude to be decreasing if needed (common standard)
    # if ds['lat'][0] < ds['lat'][-1]:
    #     print("Reindexing latitude to decreasing order...")
    #     ds = ds.reindex(lat=list(reversed(ds['lat'])))

    print("Xarray Dataset created successfully:")
    print(ds)
except KeyError as e:
    print(f"\nError: Column '{e}' not found in DataFrame. Needed for setting index.")
    print("Please ensure your DataFrame has 'time', 'lat', 'lon' columns.")
    exit()
except Exception as e:
    print(f"\nAn error occurred during DataFrame to Xarray conversion: {e}")
    exit()


# --- Step 3: Perform Spatial Aggregation using groupby_bins ---
print(f"\nAggregating data to {lat_res_out}° x {lon_res_out}° grid using groupby_bins...")

try:
    # 3a: Determine the overall extent and create output grid *boundaries*
    min_lat, max_lat = ds['lat'].min().item(), ds['lat'].max().item()
    min_lon, max_lon = ds['lon'].min().item(), ds['lon'].max().item()

    # Calculate the boundaries aligned with the output resolution
    # Use floor for min edge, ceil for max edge, ensuring they align with the grid
    # Add a small epsilon to max extent before ceil to handle points exactly on boundary
    epsilon = 1e-9
    lat_min_bnd = np.floor(min_lat / lat_res_out) * lat_res_out
    lat_max_bnd = np.ceil((max_lat + epsilon) / lat_res_out) * lat_res_out
    lon_min_bnd = np.floor(min_lon / lon_res_out) * lon_res_out
    lon_max_bnd = np.ceil((max_lon + epsilon) / lon_res_out) * lon_res_out

    # Create the bin edges for grouping
    # Use arange up to max_bnd + resolution/2 to ensure the last bin edge is included
    lat_bins = np.arange(lat_min_bnd, lat_max_bnd + lat_res_out / 2, lat_res_out)
    lon_bins = np.arange(lon_min_bnd, lon_max_bnd + lon_res_out / 2, lon_res_out)

    # 3b: Calculate the desired output grid *centroids* (labels for the bins)
    # These will become the new coordinates in the aggregated dataset
    lat_centroids = lat_bins[:-1] + lat_res_out / 2.0
    lon_centroids = lon_bins[:-1] + lon_res_out / 2.0

    print(f"Latitude bins: {lat_bins}")
    print(f"Latitude centroids: {lat_centroids}")
    print(f"Longitude bins: {lon_bins}")
    print(f"Longitude centroids: {lon_centroids}")

    # 3c: Perform the aggregation
    # Group by latitude bins first, calculate the mean for each bin,
    # then group the result by longitude bins and calculate the mean.
    # Use the calculated centroids as the coordinate labels for the resulting bins.
    # 'right=False' means bins are [left, right), matching np.floor logic.
    ds_agg = ds.groupby_bins('lat', bins=lat_bins, labels=lat_centroids, right=False).mean()
    ds_agg = ds_agg.groupby_bins('lon', bins=lon_bins, labels=lon_centroids, right=False).mean()

    # Rename the coordinate dimensions from 'lat_bins'/'lon_bins' back to 'lat'/'lon'
    ds_agg = ds_agg.rename({'lat_bins': 'lat', 'lon_bins': 'lon'})
   

    print("Aggregation complete.")
    print("Aggregated Dataset:")
    print(ds_agg)

except Exception as e:
    print(f"\nAn error occurred during aggregation with groupby_bins: {e}")
    print("Check if latitude/longitude coordinates are monotonic (sorted).")
    exit()


# --- Step 4: Convert back to DataFrame and save ---
print("\nConverting aggregated Dataset back to DataFrame...")
# Use .reset_index() to turn the coordinates ('time', 'lat', 'lon')
# back into columns for the CSV/Parquet output.
df_agg = ds_agg.to_dataframe().reset_index()

# Optional: Drop rows where the aggregated value is NaN
# This happens if an output grid cell contained no input data points.
nan_count_before = df_agg['waterstorage'].isnull().sum()
if nan_count_before > 0:
    print(f"Dropping {nan_count_before} rows with NaN 'waterstorage' values (empty grid cells).")
    df_agg = df_agg.dropna(subset=['waterstorage'])

print(f"\nSaving aggregated data...")
try:
    # Save to CSV
    print(f"Saving to CSV: {output_csv_file}")
    df_agg.to_csv(output_csv_file, index=False, float_format='%.5f') # Control float precision

    # Save to Parquet (often more efficient for large data)
    print(f"Saving to Parquet: {output_parquet_file}")
    df_agg.to_parquet(output_parquet_file, index=False)

    df_gldas = df_agg.copy()

    print("Aggregated data saved successfully.")
    print("\nFinal DataFrame snippet:")
    print(df_agg.head())

except Exception as e:
    print(f"\nAn error occurred while saving the output files: {e}")

print("\nScript finished.")

Starting script...
Output CSV: ../../output/downsampled_GLDAS_5deg_water_aligned.csv
Output Parquet: ../../output/downsampled_GLDAS_5deg_water_aligned.parquet
Input resolution: 0.25° x 0.25° (assumed)
Output resolution: 5.0° x 5.0°

Using pre-loaded DataFrame...
Converting 'time' column to datetime...
DataFrame ready.
DataFrame head:
        time     lat      lon  SoilMoi0_10cm_inst  SoilMoi10_40cm_inst  \
0 2000-01-01  82.875  -67.625           32.961132            163.73515   
1 2000-01-01  64.875 -103.875           26.860878             67.51416   
2 2000-01-01  64.875 -104.125           26.629623             68.30016   
3 2000-01-01  64.875 -104.375           26.186129             69.08916   
4 2000-01-01  64.875 -104.625           25.803741             69.82496   

   SoilMoi40_100cm_inst  SoilMoi100_200cm_inst  waterstorage  
0             336.96610              456.84467    355.720494  
1              86.47700              300.45468    187.640608  
2              85.29900       

In [6]:

# --- 1. Prepare Data ---
# Convert time columns to datetime objects
df_grace['time'] = pd.to_datetime(df_grace['time'])
df_gldas['time'] = pd.to_datetime(df_gldas['time'])

# Rename waterstorage columns for clarity after merge
df_grace = df_grace.rename(columns={'waterstorage': 'waterstorage_grace'})
df_gldas = df_gldas.rename(columns={'waterstorage': 'waterstorage_gldas'})
print(df_grace.head())
print(df_gldas.head())

# Select relevant columns from GLDAS (optional, keeps df smaller)
df_gldas_subset = df_gldas[['lon', 'lat', 'time', 'waterstorage_gldas']]

# --- 3. Merge ---
# Merge based on location and time
# Use 'inner' merge to only keep rows where lon, lat, and time match in both datasets
df_merged = pd.merge(df_grace, df_gldas_subset, on=['lon', 'lat', 'time'], how='inner')
print(df_merged.head())
# --- 4. Standardize ---
# Calculate mean and standard deviation for each waterstorage column *within the merged data*
grace_mean = df_merged['waterstorage_grace'].mean()
grace_std = df_merged['waterstorage_grace'].std()

gldas_mean = df_merged['waterstorage_gldas'].mean()
gldas_std = df_merged['waterstorage_gldas'].std()

# Add a small epsilon to standard deviation to prevent division by zero if std is 0
epsilon = 1e-9 

# Calculate standardized (Z-score) values
df_merged['grace_std'] = (df_merged['waterstorage_grace'] - grace_mean) / (grace_std + epsilon)
df_merged['gldas_std'] = (df_merged['waterstorage_gldas'] - gldas_mean) / (gldas_std + epsilon)

# --- 5. Combine ---
# Calculate the combined standardized value
df_merged['combined_std'] = 0.5 * df_merged['grace_std'] + 0.5 * df_merged['gldas_std']
df_merged.rename(columns={'combined_std': 'waterstorage'}, inplace=True)

# --- Display Results ---
print("Merged and Standardized Data:")
# Show relevant columns
print(df_merged[['lon', 'lat', 'time', 'waterstorage_grace', 'waterstorage_gldas', 'grace_std', 'gldas_std', 'waterstorage']].head())
# Save the merged DataFrame to CSV
output_merged_csv_file = os.path.join(output_dir, 'merged_standardized_data_0.5_0.5.csv')
df_merged.to_csv(output_merged_csv_file, index=False, float_format='%.5f') # Control float precision
# Save the merged DataFrame to Parquet
output_merged_parquet_file = os.path.join(output_dir, 'merged_standardized_data_0.5_0.5.parquet')
df_merged.to_parquet(output_merged_parquet_file, index=False)
print(f"Merged data saved to CSV: {output_merged_csv_file}")
print(f"Merged data saved to Parquet: {output_merged_parquet_file}")

     lon   lat       time  waterstorage_grace
0 -127.5  47.5 2002-04-01        0.000000e+00
1 -127.5  47.5 2002-05-01        4.336809e-19
2 -127.5  47.5 2002-06-01       -2.602085e-18
3 -127.5  47.5 2002-07-01       -1.040834e-17
4 -127.5  47.5 2002-08-01        3.469447e-18
       lon   lat       time  SoilMoi0_10cm_inst  SoilMoi10_40cm_inst  \
900 -142.5  57.5 2000-01-01           83.875445           254.035585   
901 -142.5  57.5 2000-02-01           83.731920           254.116095   
902 -142.5  57.5 2000-03-01           81.630962           254.100155   
903 -142.5  57.5 2000-04-01           80.474056           249.098585   
904 -142.5  57.5 2000-05-01           79.682159           240.479580   

     SoilMoi40_100cm_inst  SoilMoi100_200cm_inst  waterstorage_gldas  
900            486.607762             847.055927          611.809402  
901            502.109110             847.055927          616.464707  
902            506.933877             847.055927          617.804698  
903    